# Задача 10. Рекомендательный сервис для определения оптимальных мест размещения постаматов в рамках проекта "Московский постамат"

**Задача:** Разработать сервис для предоставления рекомендаций по оптимальному размещению постаматов возле городских киосков или на городской территории с точки зрения потенциальной востребованности постамата у жителей города (далее – сервис). 


Сервис позволит повысить востребованность устанавливаемых постаматов, а также обеспечить доступность удобного и безопасного способа доставки для наибольшего количества москвичей.

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
data = pd.read_csv('preprocessed_dataset.csv', index_col=0)

/var/folders/f5/f6tlzch97_542qvp_7hrb6880000gq/T/ipykernel_24096/239425083.py:1: DtypeWarning: Columns (1,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('preprocessed_dataset.csv', index_col=0)


In [14]:
# Отнормируем пассажиропоток
data['passenger_flow_n'] = (data['passenger_flow'] - data['passenger_flow'].min()) /\
    (data['passenger_flow'].max() - data['passenger_flow'].min())

In [53]:
pvz_info = pd.read_csv('pvz_merged.csv')

## 2. Обучение моделей

Основой для дальнейшего построения покрытия города будут оценки потенциальных мест, посчитанные той или иной моделью. Ожидается ранжирование точек по убыванию коэффициентов, посчитанных моделью.

### 2.1 Экспертная, или математическая модель

Основная идея данной модели заключается в наибольшей интерпретируемости – она учитывает в себе ряд метрик, при этом их комбинация построена исходя из общих соображений, выработанных эмпирическим путем.

In [ ]:
all_candidates[""]

Учитываем приоритет на типы локаций.

In [20]:
type_to_indicator = {
  "kiosk": 0.5,
  "multifunctional": 0.4,
  "library": 0.3,
  "culture_house": 0.2,
  "sports": 0.1,
  "house": 0,
  "underground": 0  
}

1. Учёт расстояния до станции нормируем в отрезок от 0 до 0.9, используя функцию сигмоиды:
$$\dfrac{1}{1 + \exp\left(\alpha \cdot dist + \beta\right)},$$
где $\alpha$ и $\beta$ подбирались из таких соображений, чтобы к расстоянию 2км до ближайшей станции вклад этого слагаемого уходил в ноль.

2. Количество пвз конкурентов сравниваем с максимальным, которое располагается в окрестности 500 метров до потенциального расположения постамата – нормируем количество пвз в окрестности на максимальное зафиксированное количество пвз.

3. Аналогично поступаем со станциями метро/МЦД/МЦК.

4. Аналогично поступаем с покрытием, учитывая с коэффициентом 1.5, как более значимый.

5. Пассажиропоток нормируем в отрезок от 0 до 1 и учитываем с коэффициентом 0.5, как менее значимый.

6. Тип здания учитываем в соответствии с ТЗ.

In [18]:
all_candidates = data.copy()

In [21]:
all_candidates['type_ind'] = all_candidates['type'].apply(lambda x: type_to_indicator[x])

In [39]:
all_candidates['indicator'] = all_candidates['pvz_cnt'] / all_candidates['pvz_cnt'].max() + \
    all_candidates['station_cnt'] / all_candidates['station_cnt'].max() + \
    1 / (1 + np.exp(2.6 * all_candidates['station_dist'] - 2.2)) + \
    all_candidates['passenger_flow_n'] * 0.5 + all_candidates['type_ind'] +\
    all_candidates['coverage'] / all_candidates['coverage'].max() * 1.5

In [40]:
all_candidates['indicator'] = (all_candidates['indicator'] - all_candidates['indicator'].min()) /\
    (all_candidates['indicator'].max() - all_candidates['indicator'].min()) * 10

In [41]:
all_candidates['indicator'].max(), all_candidates['indicator'].min()

(10.0, 0.0)

In [42]:
all_candidates.to_csv('./math_model_result.csv')

""


### 2.2 Двустадийная модель

Будем классифицировать потенциальные точки для расстановки постаматов, обучаясь на уже расставленных постаматах "конкурентов".

Далее те места, которые обученный классификатор предсказывает, как относящиеся к классу постаматов, ранжируем в соответствии с известной математической формулой.

In [119]:
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style='dark', font_scale=1.7)

In [123]:
all_candidates.head(2)

,title,description,type,lat,lon,area,district,address,"square, m2",year,...,population,pvz_cnt,station_cnt,station_dist,passenger_flow,dist_to_center,coverage,indicator,passenger_flow_n,type_ind
0,"ГБУК г. Москвы «ОКЦ ЮАО», Библиотека №140",Библиотека,library,55.583967,37.685992,Южный административный округ,Бирюлёво Восточное,"Российская Федерация, город Москва, внутригоро...",NaN,NaN,...,NaN,1,0,5.000000,-0.512940,19.130877,14856.0,2.171303,0.000000,0.3
1,"ГБУК г. Москвы «ОКЦ ЮАО», Библиотека №166 им. ...",Библиотека,library,55.708619,37.586619,Южный административный округ,Донской,"Российская Федерация, город Москва, внутригоро...",NaN,NaN,...,NaN,3,2,0.059873,1.180246,5.196476,7980.0,5.979018,0.325828,0.3


Обогатим датасет ПВЗ примерами мест, где ПВЗ не расположены – это будут отрицательные примеры для модели классификации. При этом будем брать наиболее далеко расположенные от постаматов места.

In [176]:
train = all_candidates.copy()
train['pvz_dist'] = train['station_dist']

In [177]:
test = pvz_info.copy()
test = test.append(train.copy().sort_values(by='pvz_dist', ascending=False).iloc[:len(test),:])[pvz_info.columns]
test = test[list(set(pvz_info.columns) & set(train.columns))[1:]]

train = train.sort_values(by='pvz_dist', ascending=False).iloc[len(test):,:]
train = train[list(set(pvz_info.columns) & set(train.columns))[1:]]
test['marker'] = [1] * len(pvz_info) + [0] * len(pvz_info)

In [191]:
from sklearn.model_selection import train_test_split

In [192]:
X_train, X_test, y_train, y_test = train_test_split(
    test.drop('marker', axis=1), test['marker'], test_size=0.33, random_state=42)

In [194]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [170]:
accuracy = accuracy_score(y_test, clf.predict(X_test))
print(f'Accuracy: {accuracy:.3f}')

precision = precision_score(y_test, clf.predict(X_test))
print(f'Precision = {precision:.3f}')

recall = recall_score(y_test, clf.predict(X_test))
print(f'Recall = {recall:.3f}')

f1 = f1_score(y_test, clf.predict(X_test))
print(f'F1 = {f1:.3f}')

Accuracy: 0.872
Precision = 0.901
Recall = 0.782
F1 = 0.798


Далее будем использовать ответ классификатора, как потенциальные места для расположения постамата. 

In [204]:
preds = clf.predict(train)
potential_places = train[preds == 1]

In [206]:
m2_candidates = pd.merge(potential_places, all_candidates, how='inner', on=['lat', 'lon'])

In [209]:
m2_candidates.iloc[:,2:].head(2)

,lon,lat,title,description,type,area,district,address,"square, m2",year,...,population,pvz_cnt,station_cnt,station_dist,passenger_flow,dist_to_center_y,coverage_y,indicator,passenger_flow_n,type_ind
0,37.854562,55.709885,NaN,Многоквартирный жилой дом,house,Восточный административный округ,Косино-Ухтомский,"ул. Косинская Б., д. 12, Москва",10627.8,1972.0,...,432.0,1,0,5.0,-0.51294,15.483746,26889.0,2.376749,0.0,0.0
1,37.555149,55.745968,NaN,Многоквартирный жилой дом,house,Западный административный округ,Дорогомилово,"ул. Дорогомиловская Б., д. 9, Москва",10668.0,1954.0,...,414.0,1,0,5.0,-0.51294,4.057709,38412.0,3.288607,0.0,0.0


In [213]:
m2_candidates['indicator'] = m2_candidates['pvz_cnt'] / m2_candidates['pvz_cnt'].max() + \
    m2_candidates['station_cnt'] / m2_candidates['station_cnt'].max() + \
    1 / (1 + np.exp(2.6 * m2_candidates['station_dist'] - 2.2)) + \
    m2_candidates['passenger_flow_n'] * 0.5 + m2_candidates['type_ind'] +\
    m2_candidates['coverage_y'] / m2_candidates['coverage_y'].max() * 1.5

In [214]:
m2_candidates.to_csv('./two_step_model_result.csv')

### 2.3 Нейронная модель

Обучим полносвязную нейронную сеть на имеющихся данных. В этом разделе она заменит этап классификации и будет отвечать на вопрос, подходит ли рассматриваемое место для того, чтобы поставить там постамат. Ранжировать подходящие точки будем математической формулой.

In [221]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [380]:
df = test.sample(frac=1)

In [381]:
x = torch.tensor(df.drop('marker', axis=1).values.astype(np.float32))
y = torch.tensor(df['marker'].values.astype(np.float32))

In [382]:
num_features = x.shape[1]

In [383]:
class ClassificationNeuralNetwork(nn.Module):
    def __init__(self, input_shape = num_features, num_classes = 10, input_channels = 1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
#             nn.Flatten(),
            nn.Linear(input_shape, 64),
            nn.ReLU(),
#             nn.Dropout(p=0.5, inplace=False),
            nn.Linear(64, 128),
            nn.ReLU(),
#             nn.Dropout(p=0.5, inplace=False),
            nn.Linear(128, num_classes),
            nn.Sigmoid()
        )
        
    def forward(self, inp):       
        out = self.model(inp)
        return out

In [384]:
model = ClassificationNeuralNetwork(num_classes=1)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

N_iter = 500

In [389]:
for t in range(N_iter):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y.unsqueeze(1))

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Используем полученную модель классификациии.

In [357]:
final_marks = model(torch.tensor(train.values.astype(np.float32)))

In [366]:
marks_np = np.array(final_marks.flatten().detach().numpy())

Зададим border_value, как 0.7, чтобы отсеять наиболее хорошие по результатам модели места.

In [394]:
border_value = 0.7

In [395]:
potential_places = train[marks_np > border_value]

In [396]:
m_NN_candidates = pd.merge(potential_places, all_candidates, how='inner', on=['lat', 'lon'])

In [397]:
m_NN_candidates['indicator'] = m_NN_candidates['pvz_cnt'] / m_NN_candidates['pvz_cnt'].max() + \
    m_NN_candidates['station_cnt'] / m_NN_candidates['station_cnt'].max() + \
    1 / (1 + np.exp(2.6 * m_NN_candidates['station_dist'] - 2.2)) + \
    m_NN_candidates['passenger_flow_n'] * 0.5 + m_NN_candidates['type_ind'] +\
    m_NN_candidates['coverage_y'] / m_NN_candidates['coverage_y'].max() * 1.5

In [398]:
m_NN_candidates.to_csv('./black_box_model_model_result.csv')